In [1]:
%pip install transformers datasets evaluate transformers[torch] accelerate ipywidgets


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model = AutoModelForCausalLM.from_pretrained("gpt2")
# tokenizer = AutoTokenizer.from_pretrained("gpt2") #BPE

# prompt = "GPT2 is a model developed by OpenAI."

# input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# gen_tokens = model.generate(
#     input_ids,
#     do_sample=True,
#     temperature=0.9,
#     max_length=100,
# )
# gen_text = tokenizer.batch_decode(gen_tokens)[0]
# print("Generated text: ", gen_text)

In [3]:
from datasets import load_dataset

dataset = load_dataset("TayTT/REMI_set")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['file', 'tokens'],
        num_rows: 1020
    })
    test: Dataset({
        features: ['file', 'tokens'],
        num_rows: 128
    })
    valid: Dataset({
        features: ['file', 'tokens'],
        num_rows: 128
    })
})

In [5]:
from datasets import DatasetDict

# Assuming your DatasetDict is already loaded and named `data`

def int_to_char(example):
    # Converts each integer token to its corresponding character
    example['tokens'] = [str(token) for token in example['tokens']]
    return example

# Apply the transformation to each split in the DatasetDict
dataset = dataset.map(int_to_char, batched=True)

# Now `data` will have 'tokens' as characters instead of integers


In [6]:
# dataset['train'][0]
# dataset['train'][0]['file']
dataset['train'][0]['tokens']
# dataset['tokens']

'[4, 189, 294, 206, 55, 107, 126, 209, 39, 103, 127, 210, 55, 106, 132, 212, 43, 106, 127, 215, 46, 105, 126, 218, 56, 111, 125, 51, 106, 127, 4, 189, 41, 108, 133, 58, 109, 126, 190, 56, 112, 125, 191, 58, 105, 125, 56, 107, 130, 192, 51, 107, 126, 194, 50, 107, 127, 197, 55, 109, 125, 198, 48, 101, 127, 56, 104, 125, 200, 58, 109, 136, 43, 105, 135, 203, 46, 108, 128, 206, 50, 108, 127, 209, 51, 107, 157, 212, 55, 109, 126, 215, 58, 112, 127, 218, 62, 112, 128, 43, 107, 127, 4, 189, 63, 111, 127, 192, 60, 111, 127, 44, 106, 126, 195, 63, 109, 127, 197, 58, 110, 127, 43, 107, 127, 200, 63, 108, 127, 203, 56, 110, 127, 204, 41, 104, 130, 206, 55, 110, 127, 209, 56, 111, 127, 50, 108, 129, 212, 65, 111, 127, 215, 56, 110, 127, 39, 109, 136, 218, 55, 110, 127, 4, 189, 53, 109, 127, 43, 108, 125, 192, 51, 108, 127, 195, 55, 110, 136, 46, 107, 128, 198, 38, 107, 127, 201, 51, 108, 125, 36, 105, 126, 203, 34, 108, 127, 207, 33, 107, 137, 48, 108, 128, 210, 60, 110, 127, 212, 53, 110, 127, 3

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2") #BPE

In [8]:
print(type(dataset['train']['tokens']))
print(dataset['train'][0]['tokens'][:4])
print(type(dataset['train'][0]['tokens'][0]))
dataset

<class 'list'>
[4, 
<class 'str'>


DatasetDict({
    train: Dataset({
        features: ['file', 'tokens'],
        num_rows: 1020
    })
    test: Dataset({
        features: ['file', 'tokens'],
        num_rows: 128
    })
    valid: Dataset({
        features: ['file', 'tokens'],
        num_rows: 128
    })
})

In [9]:
CONTEXT_LENGTH = 500


def tokenize(piece):
    outputs = tokenizer(
        piece["tokens"],
        # turncation=False,
        max_length=CONTEXT_LENGTH,
        return_overflowing_tokens=False)

    return outputs

tokenized_datasets = dataset.map(
    tokenize, batched=True, remove_columns=dataset['train'].column_names)

tokenized_datasets

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1020
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 128
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 128
    })
})

In [10]:
from transformers import AutoTokenizer

context_length = 128
tokenizer = AutoTokenizer.from_pretrained("gpt2")

outputs = tokenizer(
    dataset["train"][:2]["tokens"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

Input IDs length: 531
Input chunk lengths: [128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 1

In [11]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

C:\Users\marle\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


GPT-2 size: 124.4M parameters


In [12]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: torch.Size([5, 500])
attention_mask shape: torch.Size([5, 500])
labels shape: torch.Size([5, 500])


In [13]:
from huggingface_hub import notebook_login

notebook_login()

In [15]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="codeparrot-ds",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=5_000,
    logging_steps=5_000,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
)

ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA or MLU devices or NPU devices or certain XPU devices (with IPEX).

In [ ]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 184.00 MiB. GPU 